In [20]:
import pandas as pd
from scipy import stats
from pydataset import data
import itertools
import env

In [2]:
index = ["Has a  Macbook", "Does not have  Macbook"]
columns = ["Codeup Student", " Not a student "]

observed = pd.DataFrame([[49, 20], [1, 30]], index=index, columns=columns)
observed

,Codeup Student,Not a student
Has a Macbook,49,20
Does not have Macbook,1,30


In [3]:
α = 0.05

In [4]:
chi2, p, degf, expected = stats.chi2_contingency(observed)
p

1.4116760526193828e-09

In [5]:
if p < α:
    print("Null hypothesis is wrong")
else:
    print("Null hypothesis is right")

Null hypothesis is wrong


In [24]:
df = data("mpg")
df = df.rename(columns={"class": "v_class"})
df["is_automatic"] = df.trans.str.startswith("a")
df.head().T

,1,2,3,4,5
manufacturer,audi,audi,audi,audi,audi
model,a4,a4,a4,a4,a4
displ,1.8,1.8,2.0,2.0,2.8
year,1999,1999,2008,2008,1999
cyl,4,4,4,4,6
trans,auto(l5),manual(m5),manual(m6),auto(av),auto(l5)
drv,f,f,f,f,f
cty,18,21,20,21,16
hwy,29,29,31,30,26
fl,p,p,p,p,p


In [25]:
categorical_columns = ["manufacturer", "model", "drv", "fl", "v_class", "is_automatic"]
categorical_columns

['manufacturer', 'model', 'drv', 'fl', 'v_class', 'is_automatic']

In [32]:
combinations = list(itertools.combinations(categorical_columns, 2))
combinations

[('manufacturer', 'model'),
 ('manufacturer', 'drv'),
 ('manufacturer', 'fl'),
 ('manufacturer', 'v_class'),
 ('manufacturer', 'is_automatic'),
 ('model', 'drv'),
 ('model', 'fl'),
 ('model', 'v_class'),
 ('model', 'is_automatic'),
 ('drv', 'fl'),
 ('drv', 'v_class'),
 ('drv', 'is_automatic'),
 ('fl', 'v_class'),
 ('fl', 'is_automatic'),
 ('v_class', 'is_automatic')]

In [33]:
alpha = 0.05
outcomes = []
for combination in combinations:
    outcome = {}
    outcome["column1"] = combination[0]
    outcome["column2"] = combination[1]

    observed = pd.crosstab(df[combination[0]], df[combination[1]])
    
    chi2, p, degf, expected = stats.chi2_contingency(observed)
    
    outcome["p"] = p
    
    if p < alpha:
        outcome["reject_null"] = True
    else:
        outcome["reject_null"] = False
    
    outcomes.append(outcome)

    
pd.DataFrame(outcomes)

,column1,column2,p,reject_null
0,manufacturer,model,0.000000e+00,True
1,manufacturer,drv,5.128094e-34,True
2,manufacturer,fl,1.062455e-13,True
3,manufacturer,v_class,5.267718e-54,True
4,manufacturer,is_automatic,9.534443e-03,True
5,model,drv,1.473953e-58,True
6,model,fl,1.854690e-07,True
7,model,v_class,9.146376e-164,True
8,model,is_automatic,5.171909e-02,False
9,drv,fl,5.845580e-01,False


In [15]:
from env import get_db_url

In [37]:
sql = """
SELECT gender, dept_name
FROM employees
JOIN dept_emp using(emp_no)
JOIN departments using(dept_no)
WHERE to_date > curdate()
AND dept_name IN ('Sales', 'Marketing')
"""

url = get_db_url("employees")

df = pd.read_sql(sql, url)
df.head().T

,0,1,2,3,4
gender,F,M,F,F,F
dept_name,Marketing,Marketing,Marketing,Marketing,Marketing


In [14]:
df.dept_name.value_counts()

Sales        37701
Marketing    14842
Name: dept_name, dtype: int64

In [39]:
merged = pd.crosstab(df.gender, df.dept_name)
merged

dept_name,Marketing,Sales
gender,,
F,5864,14999
M,8978,22702


In [40]:
p

7.4513729838250925e-06

In [41]:
chi2, p, degf, expected = stats.chi2_contingency(merged)
if p < α:
    print("Null hypothesis is wrong")
else:
    print("Null hypothesis is right")

Null hypothesis is right


In [42]:
sql = """
SELECT gender, title, emp_no
FROM employees
JOIN titles using(emp_no)
"""

In [44]:
df = pd.read_sql(sql, url)
df.head().T

,0,1,2,3,4
gender,M,F,M,M,M
title,Senior Engineer,Staff,Senior Engineer,Engineer,Senior Engineer
emp_no,10001,10002,10003,10004,10004


In [45]:
df["is_manager"] = df.title == "Manager"
df.head().T

,0,1,2,3,4
gender,M,F,M,M,M
title,Senior Engineer,Staff,Senior Engineer,Engineer,Senior Engineer
emp_no,10001,10002,10003,10004,10004
is_manager,False,False,False,False,False


In [46]:
merged = pd.crosstab(df.gender, df.is_manager)
merged

is_manager,False,True
gender,,
F,177211,13
M,266073,11


In [47]:
chi2, p, degf, expected = stats.chi2_contingency(merged)
p

0.22600394509880642

In [48]:
if p < alpha:
    print("Null hypothesis is wrong")
else: 
    print("Null hypothesis is right")

Null hypothesis is right
